# 0.0. Imports

In [2]:
import pickle
import inflection

import numpy   as np
import pandas  as pd
import seaborn as sn

from sklearn         import svm
from xgboost         import XGBClassifier
from ydata_profiling import ProfileReport

from sklearn    import metrics         as mt
from sklearn    import ensemble        as en
from sklearn    import linear_model    as lm
from sklearn    import preprocessing   as pp
from sklearn    import model_selection as ms

from matplotlib import pyplot          as plt

/home/felipe/repos/hackday/hackday/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1.0. Data Description

## 1.1. Loading Data

In [3]:
# load datasets
df_raw = pd.read_csv('data/train.csv')
df_country = pd.read_csv('data/country_base_2.csv')

# mergre county coordinates
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

# drop irrelevant columns
df1 = df1.drop('country', axis=1)

## 1.2. Rename Columns

In [4]:
# list columns names
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 
            'Reserva_Cancelada', 'latitude', 'longitude']

# rename columns
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

## 1.3. Data Dimensions

In [5]:
# check dimensions
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 72159
Number of cols: 17


## 1.4. Check NA

In [6]:
# check NA values
df1.isna().sum()#/len(df1)

id                                         0
classificacao_do_hotel                     0
meses_da_reserva_ate_o_check_in            0
numero_de_pernoites_reservadas             0
numero_de_hospedes                         3
regime_de_alimentacao                      0
nacionalidade                           1093
forma_de_reserva                           0
ja_se_hospedou_anterioremente              0
tipo_do_quarto_reservado                   0
reserva_feita_por_agencia_de_turismo       0
reserva_feita_por_empresa                  0
reserva_com_estacionamento                 0
reserva_com_observacoes                    0
reserva_cancelada                          0
latitude                                1093
longitude                               1093
dtype: int64

### 1.4.1. Replace NA

In [7]:
# drop NA
#df1['numero_de_hospedes'] = df1['numero_de_hospedes'].fillna(2)
df1 = df1.dropna()

## 1.5. Data Dtypes

In [8]:
# check dtypes
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                float64
longitude                               float64
dtype: object

### 1.5.1. Change Dtypes

In [9]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].astype('int')

## 1.6. Data Balancing

In [10]:
# check balancing of target variable
df1['reserva_cancelada'].value_counts(normalize=True)

0    0.62674
1    0.37326
Name: reserva_cancelada, dtype: float64

## 1.7. Pandas Profiling

In [11]:
# prof = ProfileReport(df1)
# prof.to_file(output_file='output.html')

### 1.7.1. Investigate meses_da_reserva_ate_o_check_in

In [12]:
df1[df1['meses_da_reserva_ate_o_check_in']>24].sample(10)

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
7158,60279,4 estrelas,244,3,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
35763,68741,4 estrelas,184,3,1,Café da manha,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
42622,39607,5 estrelas,87,10,2,Café da manha e jantar,United Kingdom,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,0,55378051.0,-3435973.0
31427,66482,4 estrelas,88,6,2,Café da manha,Sweden,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,1,60128161.0,18643501.0
25920,116530,4 estrelas,96,683,2,Café da manha,Brazil,Balcão,Não,Amethyst,Sim,Não,Não,Nenhuma,0,-14235004.0,-5192528.0
43846,117812,4 estrelas,170,2,4,Café da manha,Switzerland,Agência,Não,Tanzanite,Sim,Não,Não,1 a 3,0,46818188.0,8227512.0
47146,96649,4 estrelas,310,3,2,Café da manha,Sweden,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,60128161.0,18643501.0
58318,5613,5 estrelas,115,7,2,Café da manha,Spain,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,1,40463667.0,-374922.0
53810,59464,4 estrelas,153,5,3,Café da manha,France,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,1,46227638.0,2213749.0
12998,91915,4 estrelas,309,2,2,Café da manha,Germany,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,51165691.0,10451526.0


In [13]:
df1[df1['meses_da_reserva_ate_o_check_in']>60][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,757
1,1,437


### 1.7.2. Investigate numero_de_pernoites_reservadas

In [14]:
df1[df1['numero_de_pernoites_reservadas']>365].sample(10)

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
28373,13578,5 estrelas,9,869,2,Café da manha e jantar,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
28376,432,5 estrelas,2,1131,2,"Café da manha, almoco e jantar",Spain,Balcão,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
37031,68463,4 estrelas,11,816,2,Café da manha,Spain,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,1,40463667.0,-374922.0
51266,32228,5 estrelas,2,615,2,Café da manha,Spain,Balcão,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
7558,22096,5 estrelas,1,1141,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
15259,62979,4 estrelas,2,787,1,Café da manha,Spain,B2B,Não,Amethyst,Não,Sim,Não,Nenhuma,1,40463667.0,-374922.0
55242,15750,5 estrelas,16,659,2,Café da manha e jantar,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
30588,31214,5 estrelas,2,733,1,Café da manha e jantar,Brazil,Agência,Não,Pink Sapphire,Sim,Não,Não,Nenhuma,0,-14235004.0,-5192528.0
21634,8119,5 estrelas,2,389,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
69039,10035,5 estrelas,2,543,2,Café da manha,Brazil,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,-14235004.0,-5192528.0


In [15]:
df1[df1['numero_de_pernoites_reservadas']>365][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,660
1,1,375


In [16]:
df1[df1['numero_de_pernoites_reservadas']==0][['id','reserva_cancelada']].groupby('reserva_cancelada').count().reset_index()

,reserva_cancelada,id
0,0,391
1,1,12


### 1.7.3. Investigate numero_de_hospedes

In [17]:
df1[df1['numero_de_hospedes']>10]

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
75,15778,5 estrelas,3,5,1020,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,40463667.0,-374922.0
102,2436,5 estrelas,11,7,20,Café da manha,Spain,Balcão,Não,Green Emerald,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
179,73042,4 estrelas,75,6,758,Café da manha,Poland,Agência,Não,Green Emerald,Sim,Não,Não,Nenhuma,1,51919438.0,19145136.0
187,81884,4 estrelas,2,3,470,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
251,83293,4 estrelas,1,2,381,Café da manha,Spain,B2B,Sim,Amethyst,Não,Sim,Não,1 a 3,0,40463667.0,-374922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71702,52893,4 estrelas,1,1,269,Café da manha,Spain,B2B,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
71711,8940,5 estrelas,5,2,502,Café da manha,United Kingdom,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,1,55378051.0,-3435973.0
71848,20468,5 estrelas,1,1,166,Café da manha,Spain,Agência,Sim,Amethyst,Não,Sim,Sim,1 a 3,0,40463667.0,-374922.0
71975,43193,4 estrelas,2,1,780,Café da manha e jantar,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0


In [18]:
df1[df1['numero_de_hospedes']>10][['id', 'reserva_feita_por_empresa']].groupby('reserva_feita_por_empresa').count().reset_index()


,reserva_feita_por_empresa,id
0,Não,1359
1,Sim,83


In [19]:
df1[df1['numero_de_hospedes']==0][['id', 'reserva_feita_por_agencia_de_turismo']].groupby('reserva_feita_por_agencia_de_turismo').count().reset_index()


,reserva_feita_por_agencia_de_turismo,id
0,Não,33
1,Sim,68


# 2.0. Feature Engineering

In [20]:
df2 = df1#.copy()

In [21]:
# classificacao_do_hotel
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))

# ja_se_hospedou_anterioremente
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))

# reserva_feita_por_agencia_de_turismo
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))

# reserva_feita_por_empresa
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))

# reserva_com_estacionamento
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))

## 2.1. Check Dtypes After Data Transformation

In [22]:
df2.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                        int64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                float64
longitude                               float64
dtype: object

In [23]:
# transform into int
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].astype('int')
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].astype('int')
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].astype('int')
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].astype('int')
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].astype('int')

# 3.0. Data Filtering

In [24]:
df3 = df2#.copy()

In [25]:
# reserva_feita_por_agencia_de_turismo reserva_feita_por_empresa
#df3 = df3.drop(['reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa'], axis=1)

# 4.0. Data Preparation 

In [26]:
df4 = df3#.copy()

In [27]:
## frequency encoder
#tipo_do_quarto_reservado
freq_dict = df4['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
df4['tipo_do_quarto_reservado_encoded'] = df4['tipo_do_quarto_reservado'].map(freq_dict)

# nacionalidade
freq_dict = df4['nacionalidade'].value_counts(normalize=True).to_dict()
df4['nacionalidade'] = df4['nacionalidade'].map(freq_dict)

In [28]:
# dummy variables
cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']

df4_dummy = pd.get_dummies(df4[cols_dummy])

df4 = pd.concat([df4, df4_dummy], axis=1)

In [29]:
# transform lat long into radians
df4['latitude'] = df4['latitude'].apply(lambda x: np.radians(x/1000000))
df4['longitude'] = df4['longitude'].apply(lambda x: np.radians(x/1000000))

# 5.0. Feature Selection 

In [30]:
df5 = df4#.copy()

In [31]:
# drop columns
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)

In [32]:
# split dataset into features and target
X = df5.drop(['reserva_cancelada'], axis=1)
y = df5['reserva_cancelada']

In [33]:
# split dataset into train and test
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

# 6.0. Machine Learning Models

## 6.1. XGBoost

In [34]:
# model definition
xgb_model = XGBClassifier(n_estimators=1000, max_depth=20, min_child_weight=1)

# model training
xgb_model.fit(X_train,y_train.values.ravel())

# model prediction
yhat_xgb = xgb_model.predict(X_val)

In [35]:
pickle.dump(xgb_model, open('/home/felipe/repos/hackday/xgb_model.pkl', 'wb' ))

### 6.1.1. XGBoost Performance

In [36]:
precision = mt.precision_score(y_val, yhat_xgb)
recall = mt.recall_score(y_val, yhat_xgb)
f1_score = mt.f1_score(y_val, yhat_xgb, average='macro')

In [37]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96060
Recall: 0.96442
F1 Score: 0.97013


## 6.2. Logistic Regression

In [38]:
# # model definition
# knn_model = nh.KNeighborsClassifier(n_neighbors=7)

# # model training
# knn_model.fit(x_train, y_train)

# # model prediction - poder de GENERALIZACAO
# yhat_knn = knn_model.predict_proba(x_val)

### 6.2.1. Logistic Regression Performance

In [39]:
# precision = mt.precision_score(y_val, yhat_knn)
# recall = mt.recall_score(y_val, yhat_knn)
# f1_score = mt.f1_score(y_val, yhat_knn, average='macro')

In [40]:
# print('Precision: {:.5f}'.format(precision))
# print('Recall: {:.5f}'.format(recall))
# print('F1 Score: {:.5f}'.format(f1_score))

## 6.3. Gradient Boosting

In [41]:
## model definition
gdr_model = en.GradientBoostingClassifier(n_estimators=500, random_state=42, max_depth=15)

# model training
gdr_model.fit(X_train, y_train)

# model prediction
yhat_gb = gdr_model.predict(X_val)

In [42]:
pickle.dump(gdr_model, open('/home/felipe/repos/hackday/gdr_model.pkl', 'wb' ))

### 6.3.1. Gradient Boosting Performance

In [43]:
precision = mt.precision_score(y_val, yhat_gb)
recall = mt.recall_score(y_val, yhat_gb)
f1_score = mt.f1_score(y_val, yhat_gb, average='macro')

In [44]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96150
Recall: 0.96423
F1 Score: 0.97042


## 6.4. Random Forest

In [45]:
# model definition
rf_model = en.RandomForestClassifier(n_estimators=1000,
                                     criterion='gini', 
                                     random_state=42, 
                                     n_jobs=-1, 
                                     max_depth=25)

# model training
rf_model.fit(X_train, y_train)

# model prediction
yhat_rf = rf_model.predict(X_val)

In [46]:
pickle.dump(rf_model, open('/home/felipe/repos/hackday/rf_model.pkl', 'wb' ))

### 6.4.1. Random Forest Performance

In [47]:
precision = mt.precision_score(y_val, yhat_rf)
recall = mt.recall_score(y_val, yhat_rf)
f1_score = mt.f1_score(y_val, yhat_rf, average='macro')

In [48]:
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.95884
Recall: 0.96556
F1 Score: 0.96984


## 6.5. SVC

In [49]:
# # model definition
# svc_model = svm.SVC()

# # model training
# svc_model.fit(X_train, y_train)

# # model prediction
# yhat_svc = svc_model.predict(X_val)

In [50]:
#pickle.dump(rf_model, open('/home/felipe/repos/hackday/rf_model.pkl', 'wb' ))

### 6.5.1. SCV Performance

In [51]:
# precision = mt.precision_score(y_val, yhat_svc)
# recall = mt.recall_score(y_val, yhat_svc)
# f1_score = mt.f1_score(y_val, yhat_svc, average='macro')

In [52]:
# print('Precision: {:.5f}'.format(precision))
# print('Recall: {:.5f}'.format(recall))
# print('F1 Score: {:.5f}'.format(f1_score))

# 7.0. Ensemble

In [53]:
# get models predicion and create a dataframe
ensemble = pd.DataFrame([yhat_xgb, yhat_gb, yhat_rf]).T

# rename columns
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']

# voting system - find mode and assing it to a new column
moda = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda

## 7.1. Ensenmble Performance

In [54]:
# prections of ensemble model
yhat_en = ensemble['pred']

# calculate metrics
precision = mt.precision_score(y_val, yhat_en)
recall = mt.recall_score(y_val, yhat_en)
f1_score = mt.f1_score(y_val, yhat_en, average='macro')

# print results
print('Precision: {:.5f}'.format(precision))
print('Recall: {:.5f}'.format(recall))
print('F1 Score: {:.5f}'.format(f1_score))

Precision: 0.96321
Recall: 0.96631
F1 Score: 0.97193
